In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, f1_score

from imblearn.over_sampling import SMOTE
import xgboost as xgb
import lightgbm as lgb
import pickle
import joblib

import warnings
from sqlalchemy import create_engine

warnings.filterwarnings("ignore")


In [7]:
# Load dataset with feature engineering applied
db_user = "postgres"
db_password = "admin123"
db_host = "localhost"
db_port = "5432"
db_name = "test"

engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
df = pd.read_sql('SELECT * FROM employee_features', engine)

# Map target variable
df['Attrition'] = df['Attrition'].map({'Yes': 1, 'No': 0})

# Drop unnecessary columns if present
df.drop(columns=[col for col in ['EmployeeCount', 'Over18', 'StandardHours', 'EmployeeNumber'] if col in df.columns], inplace=True)

# Encode categorical variables
cat_cols = df.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col])

df.head()


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,...,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,TenureCategory,OverTimeBinary,EngagementScore
0,41,NaN,2,1102,2,1,2,1,2,0,...,8,0,1,6,4,0,5,1,0,7
1,49,NaN,1,279,1,8,1,1,3,1,...,10,3,3,10,7,1,7,0,0,4
2,37,NaN,2,1373,1,2,2,4,4,1,...,7,3,3,0,0,0,0,2,0,5
3,33,NaN,1,1392,1,3,4,1,4,0,...,8,3,3,8,7,3,0,0,0,6
4,27,NaN,2,591,1,2,1,3,1,1,...,6,3,3,2,2,2,2,2,0,5


In [13]:
# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data
x_train, x_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

# Handle imbalance with SMOTE
smote = SMOTE(random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)


ValueError: Input y contains NaN.